In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization
from hloc import colmap_for_habitat, triangulation, localize_sfm, visualization
import h5py
import os
import numpy as np
from collections import defaultdict
import pycolmap
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import math
import g2o
from enum import Enum
import cv2
from threading import Lock
from hloc.utils.parsers import (
    parse_image_lists_with_intrinsics, parse_retrieval, names_to_pair)
from tqdm import tqdm
import json
from time import perf_counter
import time

[08/14/2021 17:20:27 INFO] Generating new fontManager, this may take some time...


## Setup
In this notebook, we will run SfM reconstruction from scratch on a set of images. We choose the [South-Building dataset](https://openaccess.thecvf.com/content_cvpr_2013/html/Hane_Joint_3D_Scene_2013_CVPR_paper.html) - we will download it later. First, we define some paths.

In [2]:
dataset = Path('/datasets/Habitat/')
images_path = dataset / 'extracted_HPointLoc/1LXtFkjw3qL_point0/images'

outputs = Path('/datasets/Habitat/Hierarchical_Localization_outputs/sfm/')
# sfm_pairs = outputs / 'pairs_exhaustive_base-around.txt'
# sfm_pairs = outputs / 'pairs_exhaustive.txt'
sfm_dir = outputs / 'sfm_superpoint+superglue'
# sfm_pairs = dataset / 'NetVLAD/NetVLAD_database.txt' 
loc_pairs = dataset / 'FBoW/scores_FBoW_all_maps.txt'  # top 50 retrieved by NetVLAD

feature_conf = extract_features.confs['superpoint_habitat']
matcher_conf = match_features.confs['superglue']

features = feature_conf['output']
feature_filename = f"{features}.h5"
match_filename = f"{features}_{matcher_conf['output']}_{loc_pairs.stem}.h5"
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build

results = outputs / 'Habitat_hloc_superpoint+superglue_hfnet_top_3.txt'  # the result file

In [3]:
cx, cy = 128, 128
fx, fy = 128, 128
width, height = 256., 256.
fx_baseline = 300

## Extract local features

In [8]:
extract_features.main(feature_conf, images_path, outputs)

[08/06/2021 11:42:59 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1024-nms4',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
Loaded SuperPoint model
[08/06/2021 11:42:59 INFO] Found 1088 images in root /datasets/Habitat/extracted_HPointLoc/1LXtFkjw3qL_point0/images.


OSError: Unable to create file (unable to truncate a file which is already open)

## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [9]:
match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

[08/06/2021 11:43:04 INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


  1%|          | 5/952 [00:00<00:24, 38.71it/s]

Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 952/952 [00:30<00:00, 30.85it/s]

[08/06/2021 11:43:35 INFO] Finished exporting matches.
Среднее время рабоыы SuperGlue: 0.02892
Медианное время рабоыы SuperGlue: 0.02949


## Defining G2O class

In [3]:
class BundleAdjustment(g2o.SparseOptimizer):
    def __init__(self, delta=np.sqrt(5.991), gain_threshold=1e-6):
        super().__init__()

        # Higher confident (better than CHOLMOD, according to 
        # paper "3-D Mapping With an RGB-D Camera")
        solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
        solver = g2o.OptimizationAlgorithmLevenberg(solver)
        super().set_algorithm(solver)
        super().set_verbose(True)

        # Convergence Criterion
        terminate = g2o.SparseOptimizerTerminateAction()
#         terminate.set_gain_threshold(1e-6)
        terminate.set_gain_threshold(gain_threshold)
        super().add_post_iteration_action(terminate)

        # Robust cost Function (Huber function) delta
#         self.delta = np.sqrt(5.991) 
        self.delta = delta
        self.aborted = False

    def optimize(self, max_iterations=10 ):
        super().initialize_optimization()
        super().optimize(max_iterations)
        try:
            return not self.aborted
        finally:
            self.aborted = False

    def add_pose(self, pose_id, pose, cam, fixed=False):
        sbacam = g2o.SBACam(
            pose.orientation(), pose.position())
        sbacam.set_cam(
            cam.fx, cam.fy, cam.cx, cam.cy, cam.baseline)

        v_se3 = g2o.VertexCam()
        v_se3.set_id(pose_id * 2)
        v_se3.set_estimate(sbacam)
        v_se3.set_fixed(fixed)
        super().add_vertex(v_se3) 

    def add_point(self, point_id, point, fixed=False, marginalized=True):
        v_p = g2o.VertexSBAPointXYZ()
        v_p.set_id(point_id * 2 + 1)
        v_p.set_marginalized(marginalized)
        v_p.set_estimate(point)
        v_p.set_fixed(fixed)
        super().add_vertex(v_p)

    def add_edge(self, id, point_id, pose_id, meas):
        if meas.is_stereo():
            edge = self.stereo_edge(meas.xyx)
        elif meas.is_left():
            edge = self.mono_edge(meas.xy)
        elif meas.is_right():
            edge = self.mono_edge_right(meas.xy)

        edge.set_id(id)
        edge.set_vertex(0, self.vertex(point_id * 2 + 1))
        edge.set_vertex(1, self.vertex(pose_id * 2))
        kernel = g2o.RobustKernelHuber(self.delta)
        edge.set_robust_kernel(kernel)
        super().add_edge(edge)

    def stereo_edge(self, projection, information=np.identity(3)):
        e = g2o.EdgeProjectP2SC()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def mono_edge(self, projection, 
            information=np.identity(2) * 0.5):
        e = g2o.EdgeProjectP2MC()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def mono_edge_right(self, projection, 
            information=np.identity(2) * 0.5):
        e = g2o.EdgeProjectP2MCRight()
        e.set_measurement(projection)
        e.set_information(information)
        return e

    def get_pose(self, id):
        return self.vertex(id * 2).estimate()

    def get_point(self, id):
        return self.vertex(id * 2 + 1).estimate()

    def abort(self):
        self.aborted = True

In [4]:
class Camera(object):
    def __init__(self, fx, fy, cx, cy, width, height, baseline):
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.baseline = baseline

        self.intrinsic = np.array([
            [fx, 0, cx], 
            [0, fy, cy], 
            [0, 0, 1]])

#         self.frustum_near = frustum_near
#         self.frustum_far = frustum_far

        self.width = width
        self.height = height
        
    def compute_right_camera_pose(self, pose):
        pos = pose * np.array([self.baseline, 0, 0])
        return g2o.Isometry3d(pose.orientation(), pos)

In [5]:
class GraphMeasurement(object):
    def __init__(self):
        self.keyframe = None
        self.mappoint = None

    @property
    def id(self):
        return (self.keyframe.id, self.mappoint.id)

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, rhs):
        return (isinstance(rhs, GraphMeasurement) and
            self.id == rhs.id)

In [6]:
class Measurement(GraphMeasurement):
    
    Source = Enum('Measurement.Source', ['TRIANGULATION', 'TRACKING', 'REFIND'])
    Type = Enum('Measurement.Type', ['STEREO', 'LEFT', 'RIGHT'])

    def __init__(self, type, source, keypoints):
        super().__init__()

        self.type = type
        self.source = source
        self.keypoints = keypoints
        self.view = None    # mappoint's position in current coordinates frame

        self.xy = np.array(self.keypoints[0].pt)
        if self.is_stereo():
            self.xyx = np.array([
                *keypoints[0].pt, keypoints[1].pt[0]])

        self.triangulation = (source == self.Source.TRIANGULATION)

    def get_keypoint(self, i=0):
        return self.keypoints[i]

    def get_keypoints(self):
        return self.keypoints

    def is_stereo(self):
        return self.type == Measurement.Type.STEREO
    def is_left(self):
        return self.type == Measurement.Type.LEFT
    def is_right(self):
        return self.type == Measurement.Type.RIGHT

    def from_triangulation(self):
        return self.triangulation
    def from_tracking(self):
        return self.source == Measurement.Source.TRACKING
    def from_refind(self):
        return self.source == Measurement.Source.REFIND

In [7]:
class GraphMapPoint(object):
    def __init__(self):
        self.id = None
        self.meas = dict()
        self._lock = Lock()

    def __hash__(self):
        return self.id

    def __eq__(self, rhs):
        return (isinstance(rhs, GraphMapPoint) and 
            self.id == rhs.id)
    def __lt__(self, rhs):
        return self.id < rhs.id
    def __le__(self, rhs):
        return self.id <= rhs.id

    def measurements(self):
        with self._lock:
            return self.meas.keys()

    def keyframes(self):
        with self._lock:
            return self.meas.values()

    def add_measurement(self, m):
        with self._lock:
            self.meas[m] = m.keyframe

    def remove_measurement(self, m):
        with self._lock:
            try:
                del self.meas[m]
            except KeyError:
                pass

In [8]:
class MapPoint(GraphMapPoint):
    _id = 0
    _id_lock = Lock()

    def __init__(self, position, covariance=np.identity(3) * 1e-4):
        super().__init__()

        with MapPoint._id_lock:
            self.id = MapPoint._id
            MapPoint._id += 1

        self.position = position
        self.covariance = covariance
        # self.owner = None

        self.count = defaultdict(int)

    def update_position(self, position):
        self.position = position
    def update_normal(self, normal):
        self.normal = normal
    def update_descriptor(self, descriptor):
        self.descriptor = descriptor
    def set_color(self, color):
        self.color = color

    def is_bad(self):
        with self._lock:
            status =  (
                self.count['meas'] == 0
                or (self.count['outlier'] > 20
                    and self.count['outlier'] > self.count['inlier'])
                or (self.count['proj'] > 20
                    and self.count['proj'] > self.count['meas'] * 10))
            return status

    def increase_outlier_count(self):
        with self._lock:
            self.count['outlier'] += 1
    def increase_inlier_count(self):
        with self._lock:
            self.count['inlier'] += 1
    def increase_projection_count(self):
        with self._lock:
            self.count['proj'] += 1
    def increase_measurement_count(self):
        with self._lock:
            self.count['meas'] += 1

In [10]:
fx, fy = 128., 128.
cx, cy = 128., 128.
width, height = 256, 256
fx_baseline = 300
baseline = fx_baseline/fx
cam = Camera(
    fx, fy, cx, cy, 
    width, height, 
    baseline)

# Getting poses for query images

In [11]:
def quaternion_to_rotation_matrix(quaternion_wxyz):
    r = R.from_quat([quaternion_wxyz[1], quaternion_wxyz[2], quaternion_wxyz[3], quaternion_wxyz[0]])
    matrix = r.as_matrix()
    matrix[:3,2] = -matrix[:3,2]
    matrix[:3,1] = -matrix[:3,1]
    return matrix

In [12]:
def get_point_3d(x, y, depth, fx, fy, cx, cy, cam_center_world, R_world_to_cam, w_in_quat_first = True):
    if depth <= 0:
        return 0
    new_x = (x - cx)*depth/fx
    new_y = (y - cy)*depth/fy
    new_z = depth
    coord_3D_world_to_cam = np.array([new_x, new_y, new_z], float)
    if len(R_world_to_cam) == 4:
        if w_in_quat_first:
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
        else:
            R_world_to_cam = [R_world_to_cam[3], R_world_to_cam[0], R_world_to_cam[1], R_world_to_cam[2]]
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
    coord_3D_cam_to_world = np.matmul(matrix, coord_3D_world_to_cam) + cam_center_world
    return coord_3D_cam_to_world

In [13]:
def yaw_pitch_roll_from_quaternion(quaternion, w_first=False):
    if not(w_first):
        r = R.from_quat(quaternion)
    else:
        r = R.from_quat([quaternion[1], quaternion[2], quaternion[3] ,quaternion[0]])
    return r.as_euler('yxz', degrees=True)

In [15]:
# loc_pairs = '/datasets/Habitat/NetVLAD.txt'
retrievals = parse_retrieval(loc_pairs)

In [16]:
retrievals

defaultdict(list,
            {'17DRP5sb8fy_point0_query_0000': [{'17DRP5sb8fy_point0_database_0004': '0.053231'},
              {'17DRP5sb8fy_point1_database_0005': '0.018023'},
              {'17DRP5sb8fy_point0_database_0003': '0.015076'}],
             '17DRP5sb8fy_point0_query_0001': [{'17DRP5sb8fy_point0_database_0005': '0.038622'},
              {'17DRP5sb8fy_point0_database_0004': '0.011990'},
              {'17DRP5sb8fy_point0_database_0000': '0.010797'}],
             '17DRP5sb8fy_point0_query_0002': [{'17DRP5sb8fy_point0_database_0000': '0.021999'},
              {'qoiz87JEwZ2_point7_database_0005': '0.011651'},
              {'17DRP5sb8fy_point0_database_0005': '0.011509'}],
             '17DRP5sb8fy_point0_query_0003': [{'17DRP5sb8fy_point0_database_0002': '0.059054'},
              {'8WUmhLawc2A_point9_database_0004': '0.019039'},
              {'PuKPg4mmafe_point62_database_0000': '0.013454'}],
             '17DRP5sb8fy_point0_query_0004': [{'17DRP5sb8fy_point0_database_

In [17]:
a = []
for query in retrievals.keys():
    for item in retrievals[query]:
#         print(list(item.items()))
        a.append(list(item.items())[0][1])
np.unique(a)

array(['0.005004', '0.005085', '0.005131', ..., '0.663803', '0.669523',
       '0.691338'], dtype='<U8')

In [18]:
path_to_hdf5_datasets = '/datasets/Habitat/HPointLoc'
# keypoints_file = h5py.File(str(outputs / feature_filename), 'r')
# match_filename = 'feats-superpoint-n4096-r1024-nms4_matches-superglue_NetVLAD_query_top_3.h5'
# match_file = h5py.File(str(outputs / match_filename), 'r')
keypoints_file = h5py.File(str(dataset / "FBoW/keypoints_all_maps.h5"), 'r')
match_file = h5py.File(str(dataset / "FBoW/matches_all_maps.h5"), 'r')

In [19]:
fx, fy = 128., 128.
cx, cy = 128., 128.
width, height = 256, 256
fx_baseline = 300
baseline = fx_baseline/fx

cam = Camera(
    fx, fy, cx, cy, 
    width, height, 
    baseline)

top_num_db_image = 1
save_images = False

In [211]:
# out = cv2.VideoWriter('/home/NetVLAD_wrong.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, (256, 256))

In [147]:
top_num_db_image = 1

In [46]:
# output_folder_for_images = '/home/for_video'
# os.makedirs(output_folder_for_images, exist_ok=True)
# save_images = True

In [20]:
# import PIL
# from PIL import Image
# from PIL import ImageFont, ImageDraw

In [206]:
# fontsize = 13
# font = ImageFont.truetype("DejaVuSans.ttf", fontsize)

In [20]:
# import faiss
# from sklearn.metrics.pairwise import cosine_similarity
import torch
from hloc.utils.base_model import dynamic_load
from hloc import matchers
confs = {
    'superglue': {
        'output': 'matches-superglue',
        'model': {
            'name': 'superglue',
            'weights': 'outdoor',
            'sinkhorn_iterations': 50,
        },
    },
    'NN': {
        'output': 'matches-NN-mutual-dist.7',
        'model': {
            'name': 'nearest_neighbor',
            'mutual_check': True,
            'distance_threshold': 0.7,
        },
    }
}

In [21]:
conf = confs['superglue']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Model = dynamic_load(matchers, conf['model']['name'])
model = Model(conf['model']).eval().to(device)

Loaded SuperGlue model ("outdoor" weights)


In [68]:
def divide_func(baseline, min_depth_disp, x):    
    return np.where(x > 0,(baseline * min_depth_disp)/x, 0) 
def add_noise(x, sigma):    
    return np.where(x > 0, x + np.random.normal(0, sigma), 0)
threshold = 0.05

def noise_depth_map(depth_image, fx_baseline, min_depth):
#     depth_image = np.squeeze(depth_image)*10
    depth_image = depth_image*10
    disparity = divide_func(fx_baseline, min_depth, depth_image)
    disp_with_noise = add_noise(disparity, disparity.min()*threshold)
    depth_with_noise = divide_func(fx_baseline, min_depth, disp_with_noise)
    return depth_with_noise

In [74]:
path_to_monodepth = '/datasets/Habitat/monodepth2_Habiat_val/'

In [82]:
a = np.load(os.path.join(path_to_monodepth, '1LXtFkjw3qL_point10_database_0003_depth.npy'))
a = np.squeeze(np.load(os.path.join(path_to_monodepth, '1LXtFkjw3qL_point10_database_0003'+"_depth.npy")), axis=0)
a = np.squeeze(a, axis = 0)
a = cv2.resize(a, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
a.shape

(256, 256)

In [24]:
# G2O

save_images = False

metrics = {
    "(5m, 20°)"   : 0, 
    "(1m, 10°)"   : 0,
    "(0,5m, 5°)"  : 0,
    "(0,25m, 2°)" : 0,
    "(5m)"        : 0,
    "(1m)"        : 0,
    "(0,5m)"      : 0,
    "(0,25m)"     : 0
}

optimizer = BundleAdjustment()
results = {}

max_angle_error = 0
count = 0
inference_times = []
for query_image, values in tqdm(retrievals.items()):
#     if query_image != '1LXtFkjw3qL_point5_query_0000':
#         continue
    query_image = query_image.rstrip('.png')
#     print(query_image)
    measurements = []
    hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
    
    map_name_query = query_image.split('_')[0]
    if map_name_query == "1LXtFkjw3qL":
         map_name_query += "_point0"
    if map_name_query == "1pXnuDYAj8r":
        map_name_query += "_point0"
    if map_name_query == "5LpN3gDmAk7":
        map_name_query += "_point0"
    if map_name_query == "5q7pvUzZiYa":
        map_name_query += "_point1"
    
    hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, map_name_query, hdf5_filename_query), 'r')
    
    num_query = int(query_image.split('_')[-1])
    
    num_point_query = query_image.split('_')[1]
    
    is_base_query = ''
    if query_image.find('database') != -1:
        is_base_query = '_base'
    
    depth_query = 10*hdf5_file_query['depth'+is_base_query][num_query]
    
#     depth_query = noise_depth_map(hdf5_file_query['depth'+is_base_query][num_query], 300, 0.25)

#     depth_query = np.squeeze(np.load(os.path.join(path_to_monodepth, query_image+"_depth.npy")), axis=0)
#     depth_query = np.squeeze(depth_query, axis = 0)
#     depth_query = cv2.resize(depth_query, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
#     depth_query = np.expand_dims(depth_query, axis = 2)
    
#     keypoints_query - массив из пар координта (x,y), которые являются кипоинтами на query image
    keypoints_query = keypoints_file[query_image]['keypoints'].__array__()
    
    # all_matched_keypoints_ids - массив из true/false, говорящий о том, имеет ли совпадение кипоинт с соответствующим индексом
    all_matched_keypoints_ids = np.full((len(keypoints_query)), False)
    
    optimizer.clear()
    
    db_image = list(values[0].keys())[0]
    db_image = db_image.rstrip('.png')
#     print(db_image)
        
    map_name_database = db_image.split('_')[0]
    if map_name_database == "1LXtFkjw3qL":
        map_name_database += "_point0"
    if map_name_database == "1pXnuDYAj8r":
        map_name_database += "_point0"
    if map_name_database == "5LpN3gDmAk7":
        map_name_database += "_point0"
    if map_name_database == "5q7pvUzZiYa":
        map_name_database += "_point1"

    is_base_db = ''
    if db_image.find('database') != -1:
        is_base_db = '_base'

    score = list(values[0].values())[0]

    hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
    hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, map_name_database, hdf5_filename_db), 'r')

    num_db = int(db_image.split('_')[-1])

    num_point_db = db_image.split('_')[1]

    depth_db = 10*hdf5_file_db['depth'+is_base_db][num_db]
    
#     depth_db = noise_depth_map(hdf5_file_db['depth'+is_base_db][num_db], 300, 0.25)
    
#     depth_db = np.squeeze(np.load(os.path.join(path_to_monodepth, db_image+"_depth.npy")), axis=0)
#     depth_db = np.squeeze(depth_db, axis = 0)
#     depth_db = cv2.resize(depth_db, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
#     depth_db = np.expand_dims(depth_db, axis = 2)

    translation = hdf5_file_db['gps'+is_base_db][num_db]
    orientation = quaternion_to_rotation_matrix(hdf5_file_db['quat'+is_base_db][num_db])
    pose_db = np.eye(4)
    pose_db[:3,:3] = orientation
    pose_db[:3,3] = translation

    pose_query = np.eye(4)
    pose_query[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_query['quat'+is_base_query][num_query])
    pose_query[:3, 3] = hdf5_file_query['gps'+is_base_query][num_query]

    diff_pose = np.linalg.inv(pose_db) @ pose_query
    dist_diff = np.sum(diff_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(diff_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_diff = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_diff = abs(180 - abs(angle_diff-180))
    
#     if dist_diff <= 5 and angle_diff < 20:
#         metrics["(5m, 20°)"] += 1
#     if dist_diff <= 1 and angle_diff < 10:
#         metrics["(1m, 10°)"] += 1
#     if dist_diff <= 0.5 and angle_diff < 5:
#         metrics["(0,5m, 5°)"] += 1
#     if dist_diff <= 0.25 and angle_diff < 2:
#         metrics["(0,25m, 2°)"] += 1
#     if dist_diff <= 5:
#         metrics["(5m)"] += 1
#     if dist_diff <= 1:
#         metrics["(1m)"] += 1    
#     if dist_diff <= 0.5:
#         metrics["(0,5m)"] += 1
#     if dist_diff <= 0.25:
#         metrics["(0,25m)"] += 1
    
#     continue

#     if dist_diff < 1 or dist_diff > 2:
#         continue
#     if angle_diff < 5 or angle_diff > 10:
#         continue

    pose = g2o.Isometry3d(pose_db)
    optimizer.add_pose(0, pose, cam, fixed=False)

    keypoints_db = keypoints_file[db_image]['keypoints'].__array__()

    # Extract matches
#     data = {}
#     feats0, feats1 = keypoints_file[query_image], keypoints_file[db_image]
#     for k in feats1.keys():
#         data[k+'0'] = feats0[k].__array__()
#     for k in feats1.keys():
#         data[k+'1'] = feats1[k].__array__()
#     data = {k: torch.from_numpy(v)[None].float().to(device)
#             for k, v in data.items()}
#     data['image0'] = torch.empty((1, 1,)+tuple(feats0['image_size'])[::-1])
#     data['image1'] = torch.empty((1, 1,)+tuple(feats1['image_size'])[::-1])
#     pred = model(data)
#     matches_db_keypoints_ids = pred['matches0'][0].cpu().short().numpy()
    
    matches_db_keypoints_ids = match_file[query_image+"_"+db_image]['matches0'].__array__()

    # matched_keypoints_query_ids - массив из индексов кипоинтов на query image, которые имеют сопоставление
    matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]

    # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
    for keypoint_id_query in matched_keypoints_query_ids:
        if all_matched_keypoints_ids[keypoint_id_query] == False:

            # затем получаем индекс сопоставленного кипоинта на database image
            keypoint_id_database = matches_db_keypoints_ids[keypoint_id_query]

            # получаем координаты x,y для сопоставленных кипоинтов на query и database
            keypoint_xy_query = keypoints_query[keypoint_id_query]
            keypoint_xy_db = keypoints_db[keypoint_id_database]

            # получаем глубины данных кипоинтов
            depth_keypoint_query = depth_query[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
            depth_keypoint_db = depth_db[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]

            # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
            if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
                continue

            # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
            x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query

            # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
            kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
            kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
            kp_right.pt = (x_right, keypoint_xy_query[1])
            meas = Measurement(Measurement.Type.STEREO,
                   Measurement.Source.TRIANGULATION,
                   [kp_left, kp_right])

            #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
            # с кипоинтом на query image
            # Получаем 3D координату ключевой точки на database image
            global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
                                                  fx, fy, cx, cy, \
                                                  translation, hdf5_file_db['quat'+is_base_db][num_db])
            mappoint = MapPoint(global_xyz_of_keypoint)
            # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
            # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
            meas.mappoint = mappoint
            measurements.append(meas)

            # Помечаем этот кипоинт на query image тем, что в дальнейшем его не нужно рассматривать - его уже
            # добавили в измерения
            all_matched_keypoints_ids[keypoint_id_query] = True
    for i, m in enumerate(measurements):
        optimizer.add_point(i, m.mappoint.position, fixed=True)
        optimizer.add_edge(0, i, 0, m)
#     t1 = time.time()
    optimizer.optimize(10)
#     t2 = time.time()
#     inference_times.append(t2-t1)
    
    result = optimizer.get_pose(0)
#     data['g2o']['cam_pose'] = result
    
    pose_estimated = np.eye(4)
    pose_query = np.eye(4)
    
#     r = R.from_quat([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
    
#     # Now get pose from g2o optimization 
    matrix = result.matrix()[:3,:3]
    pose_estimated[:3, :3] = matrix
    pose_estimated[:3, 3] = result.position()

    # Now get pose from Global Descriptor retrieval
#     pose_estimated[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
#     pose_estimated[:3, 3] = hdf5_file_db['gps_base'][num_db]
    
#     orientation_quat_xyzw = [hdf5_file_query['quat'][num_query][1], hdf5_file_query['quat'][num_query][2],\
#                             hdf5_file_query['quat'][num_query][3], hdf5_file_query['quat'][num_query][0]]
#     r = R.from_quat(orientation_quat_xyzw)
#     matrix = r.as_matrix()
    pose_query[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_query['quat'+is_base_query][num_query])
    pose_query[:3, 3] = hdf5_file_query['gps'+is_base_query][num_query]
#     data['query']['cam_pose'] = pose_query 
#     print("GT translation:")
#     print(hdf5_file_query['gps'][num_query])
    error_pose = np.linalg.inv(pose_estimated) @ pose_query
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(180 - abs(angle_error-180))
    
    if dist_error <= 5 and angle_error < 20:
        metrics["(5m, 20°)"] += 1
    if dist_error <= 1 and angle_error < 10:
        metrics["(1m, 10°)"] += 1
    if dist_error <= 0.5 and angle_error < 5:
        metrics["(0,5m, 5°)"] += 1
    if dist_error <= 0.25 and angle_error < 2:
        metrics["(0,25m, 2°)"] += 1
    if dist_error <= 5:
        metrics["(5m)"] += 1
    if dist_error <= 1:
        metrics["(1m)"] += 1    
    if dist_error <= 0.5:
        metrics["(0,5m)"] += 1
    if dist_error <= 0.25:
        metrics["(0,25m)"] += 1
    
#     for key, value in metrics.items():
#         degree_thr = float(key.split(' ')[-2])
#         position_thr = float(key.split(' ')[0])
#         if angle_error < degree_thr and dist_error < position_thr:
#             metrics[key] += 1
    
    if dist_error < 0.25 and angle_error > 2 and save_images:
        new_image = Image.new('RGB',(2*256, 256), (250,250,250))

        db_image1 = list(retrievals[query_image+'.png'][0].keys())[0]
#         db_image2 = list(retrievals[query_image+'.png'][1].keys())[0]
#         db_image3 = list(retrievals[query_image+'.png'][2].keys())[0]
        
        image1 = Image.open(str(images_path / query_image)+'.png')
        image2 = Image.open(str(images_path / db_image1))
#         image3 = Image.open(str(images_path / db_image2))
#         image4 = Image.open(str(images_path / db_image3))
        
        metric1 = list(retrievals[query_image+'.png'][0].values())[0]
#         metric2 = list(retrievals[query_image+'.png'][1].values())[0]
#         metric3 = list(retrievals[query_image+'.png'][2].values())[0]
        
        new_image.paste(image1,(0,0))
        new_image.paste(image2,(255,0))
#         new_image.paste(image3,(2*255,0))
#         new_image.paste(image4,(3*255,0))

        draw = ImageDraw.Draw(new_image)
        
        draw.text((0, 0), query_image,(255,255,255), font=font)
        draw.text((image1.size[0], 0),"{}\n \
                                       score = {}\n \
                                       num matches = {}".format(db_image1.rstrip('.png'), 
                                                                round(float(metric1),3), 
                                                                len(matched_keypoints_query_ids)),
                  (255,255,255),font=font)
        
#         draw.text((2*image1.size[0], 0),"{}\nsim: {}".format(db_image2.rstrip('.png'), round(float(metric2),3)),(255,255,255),font=font)
#         draw.text((3*image1.size[0], 0),"{}\nsim: {}".format(db_image3.rstrip('.png'), round(float(metric3),3)),(255,255,255),font=font)

        new_image.save('{}/{:04}.jpg'.format(output_folder_for_images, count),'JPEG')
        count += 1
#     results[query_image] = {
#         'estimated_pose'   : pose_estimated.tolist(),
#         'gt_query_pose'    : pose_query.tolist(),
#         'gt_database_pose' : pose_db.tolist(),
#         'dist_error'       : float(dist_error),
#         'angle_error'      : float(angle_error),
#         'retrieved_image'  : db_image,
#         'num_matches'      : len(matched_keypoints_query_ids),
#         'retrival_conf'    : float(metric1)
#     }
    

# for key in metrics.keys():
#     metrics[key] = metrics[key] / 950

100%|██████████| 76136/76136 [1:40:06<00:00, 12.68it/s]  


In [25]:
for key in metrics.keys():
    metrics[key] = metrics[key] / len(retrievals.keys())
metrics

{'(5m, 20°)': 0.7764263948723337,
 '(1m, 10°)': 0.7482399915939897,
 '(0,5m, 5°)': 0.5710439214038037,
 '(0,25m, 2°)': 0.3567431963854156,
 '(5m)': 0.820150782809709,
 '(1m)': 0.7689529263423348,
 '(0,5m)': 0.5903777450877378,
 '(0,25m)': 0.3772065777030577}

In [29]:
with open('/datasets/Habitat/NetVLAD+SP+SG_localization_all_maps.json', 'w') as f:
    json.dump(results, f)

In [8]:
# # with open('/datasets/Habitat/HFNET+SP+SG_localization_all_maps.json', 'r') as f:
#     results = json.load(f)

In [14]:
out = cv2.VideoWriter('/home/NetVLAD_wrong.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, (256, 256))
output_folder_for_images = '/home/for_video'
os.makedirs(output_folder_for_images, exist_ok=True)
save_images = True

import PIL
from PIL import Image
from PIL import ImageFont, ImageDraw
from sklearn.metrics.pairwise import cosine_similarity


fontsize = 13
font = ImageFont.truetype("DejaVuSans.ttf", fontsize)

count = 0

In [35]:
# os.makedirs(output_folder_for_images, exist_ok=True)
save_image = False
metrics = {
    "(5m, 20°)"   : 0, 
    "(1m, 10°)"   : 0,
    "(0,5m, 5°)"  : 0,
    "(0,25m, 2°)" : 0,
    "(5m)"        : 0,
    "(1m)"        : 0,
    "(0,5m)"      : 0,
    "(0,25m)"     : 0
}
count = 0
for query_image in tqdm(results.keys()):
    
    db_image = results[query_image]['retrieved_image']
    
    map_name_database = db_image.split('_')[0]
    if map_name_database == "1LXtFkjw3qL":
        map_name_database += "_point0"
    if map_name_database == "1pXnuDYAj8r":
        map_name_database += "_point0"
    if map_name_database == "5LpN3gDmAk7":
        map_name_database += "_point0"
    if map_name_database == "5q7pvUzZiYa":
        map_name_database += "_point1"
            
    map_name_query = query_image.split('_')[0]
    if map_name_query == "1LXtFkjw3qL":
         map_name_query += "_point0"
    if map_name_query == "1pXnuDYAj8r":
        map_name_query += "_point0"
    if map_name_query == "5LpN3gDmAk7":
        map_name_query += "_point0"
    if map_name_query == "5q7pvUzZiYa":
        map_name_query += "_point1"
    
#     pose_estimated = np.array(results[query_image]['gt_database_pose'])
    db_pose = np.array(results[query_image]['gt_database_pose'])
    pose_estimated = np.array(results[query_image]['estimated_pose'])
    pose_query = np.array(results[query_image]['gt_query_pose'])
    
    error_pose_db = np.linalg.inv(db_pose) @ pose_query
    dist_error_db = np.sum(error_pose_db[:3, 3]**2) ** 0.5

    error_pose = np.linalg.inv(db_pose) @ pose_query
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(180 - abs(angle_error-180))
    
    if dist_error <= 5 and angle_error < 20:
        metrics["(5m, 20°)"] += 1
    if dist_error <= 1 and angle_error < 10:
        metrics["(1m, 10°)"] += 1
    if dist_error <= 0.5 and angle_error < 5:
        metrics["(0,5m, 5°)"] += 1
    if dist_error <= 0.25 and angle_error < 2:
        metrics["(0,25m, 2°)"] += 1
    if dist_error <= 5:
        metrics["(5m)"] += 1
    if dist_error <= 1:
        metrics["(1m)"] += 1    
    if dist_error <= 0.5:
        metrics["(0,5m)"] += 1
    elif dist_error_db <= 0.5 and save_image:
        new_image = Image.new('RGB',(2*256, 256), (250,250,250))

        db_image1 = db_image
        
        image1 = Image.open(str(images_path / Path(map_name_query) / Path("images") / Path(query_image))+'.png')
        image2 = Image.open(str(images_path / Path(map_name_database) / Path("images") / Path(db_image1))+'.png')
        
        new_image.paste(image1,(0,0))
        new_image.paste(image2,(255,0))

        draw = ImageDraw.Draw(new_image)
        
        query_x, query_y, query_z = pose_query[:3,3]
        query_x, query_y, query_z = round(query_x,1), round(query_y,1), round(query_z,1)
        
        r = R.from_matrix(pose_query[:3,:3])
        query_yaw, query_roll, query_pitch = r.as_euler('yxz', degrees=True)
        query_yaw, query_roll, query_pitch = round(query_yaw,1), round(query_roll,1), round(query_pitch,1)
        
        es_x, es_y, es_z = pose_estimated[:3,3]
        es_x, es_y, es_z = round(es_x,1), round(es_y,1), round(es_z,1)
        
        r = R.from_matrix(pose_estimated[:3,:3])
        es_yaw, es_roll, es_pitch = r.as_euler('yxz', degrees=True)
        es_yaw, es_roll, es_pitch = round(es_yaw,1), round(es_roll,1), round(es_pitch,1)
        
        db_x, db_y, db_z = db_pose[:3,3]
        db_x, db_y, db_z = round(db_x,1), round(db_y,1), round(db_z,1)
        
        r = R.from_matrix(db_pose[:3,:3])
        db_yaw, db_roll, db_pitch = r.as_euler('yxz', degrees=True)
        db_yaw, db_roll, db_pitch = round(db_yaw,1), round(db_roll,1), round(db_pitch,1)
        
        draw.text((0, 0), "{}\n {} {} {} {} {} {}".format(query_image.rstrip('.png'), \
                                                             query_x, query_y, query_z, \
                                                             query_yaw, query_pitch, query_roll),\
                  (255,255,255), font=font)
        
        draw.text((image1.size[0], 0),"{}\ng2o: {} {} {} {} {} {}\ndb: {} {} {} {} {} {}".format(
                   db_image.rstrip('.png'), \
                   es_x, es_y, es_z, \
                   es_yaw, es_pitch, es_roll,\
                   db_x, db_y, db_z, \
                   db_yaw, db_pitch, db_roll),\
                   (255,255,255),font=font)

        new_image.save('{}/{:04}.jpg'.format(output_folder_for_images, count),'JPEG')
        count += 1
    if dist_error <= 0.25:
        metrics["(0,25m)"] += 1 

100%|██████████| 76136/76136 [00:15<00:00, 4930.60it/s]


In [32]:
imgage_array = []

for filename in tqdm(sorted(os.listdir(output_folder_for_images))):
    img = cv2.imread(os.path.join(output_folder_for_images, filename))
    height, width, layers = img.shape
    size = (width,height)
    imgage_array.append(img)

out = cv2.VideoWriter('/home/video_Habitat.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)

for i in tqdm(range(len(imgage_array))):
    cv2.waitKey(10000)
    out.write(imgage_array[i])
out.release()

100%|██████████| 3527/3527 [00:04<00:00, 809.34it/s]


In [36]:
for key in metrics.keys():
    metrics[key] = metrics[key] / len(results.keys())

In [38]:
metrics

{'(5m, 20°)': 0.6429021750551644,
 '(1m, 10°)': 0.31657822843332983,
 '(0,5m, 5°)': 0.15754702112010088,
 '(0,25m, 2°)': 0.03442523904591783,
 '(5m)': 0.9565251654933278,
 '(1m)': 0.8769701586634444,
 '(0,5m)': 0.8755910475990333,
 '(0,25m)': 0.48685247451928126}

In [ ]:
# G2O

max_num_top = 1

set_of_query_images = set()

data = {
    'query'    : {},
    'database' : {},
    'g2o'      : {}
}

optimizer = BundleAdjustment()
results = {}
metrics = {
    "(5m, 20°)"   : 0,
    "(1m, 10°)"   : 0,
    "(0.5m, 5°)"  : 0,
    "(0.25m, 2°)" : 0
}
max_angle_error = 0

with open(str(loc_pairs)) as f:
    lines = f.readlines()

for line in tqdm(lines):
#     if query_image != '1LXtFkjw3qL_point5_query_0000':
#         continue
    query_image, db_image, cos_score, l2_metric = line.split(' ')
    query_image = query_image.rstrip('.png')
    db_image = db_image.rstrip('.png')
    
    if not (query_image in set_of_query_images):
        top_num_db = 0
        set_of_query_images.add(query_image)
        measurements = []
        optimizer.clear()
        hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
        hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
    
        num_query = int(query_image.split('_')[-1])
        num_point_query = query_image.split('_')[1]
    
        depth_query = 10*hdf5_file_query['depth'][num_query]
    
        # keypoints_query - массив из пар координта (x,y), которые являются кипоинтами на query image
        keypoints_query = keypoints_file[query_image]['keypoints'].__array__()
    
        # all_matched_keypoints_ids - массив из true/false, говорящий о том, имеет ли совпадение кипоинт с соответствующим индексом
        all_matched_keypoints_ids = np.full((len(keypoints_query)), False)
    
    top_num_db += 1
    
    if top_num_db  <= max_num_top:    
        num_point_db = db_image.split('_')[1]
        hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
        hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
        
        num_db = int(db_image.split('_')[-1])
        
        depth_db = 10*hdf5_file_db['depth_base'][num_db]
        
        translation = hdf5_file_db['gps_base'][num_db]
        orientation = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
        pose_44 = np.eye(4)
        pose_44[:3,:3] = orientation
        pose_44[:3,3] = translation
    
        # NetVLAD predictions
        if top_num_db == 1:
            pose_estimated[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
            pose_estimated[:3, 3] = hdf5_file_db['gps_base'][num_db]
    
        data['database']['cam_pose'] = pose_44
        pose = g2o.Isometry3d(pose_44)
        optimizer.add_pose(0, pose, cam, fixed=False)
        
        keypoints_db = keypoints_file[db_image]['keypoints'].__array__()
        
        # matches_db_keypoints_ids - массив из индексов кипоинтов на database image, которые совпадают с кипоинтом, индекс которого равен положению в массиве
        matches_db_keypoints_ids = match_file['_'.join([query_image,db_image])]['matches0'].__array__()
        
        # matched_keypoints_query_ids - массив из индексов кипоинтов на query image, которые имеют сопоставление
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        
        # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
        for keypoint_id_query in matched_keypoints_query_ids:
            if all_matched_keypoints_ids[keypoint_id_query] == False:
                
                # затем получаем индекс сопоставленного кипоинта на database image
                keypoint_id_database = matches_db_keypoints_ids[keypoint_id_query]
                
                # получаем координаты x,y для сопоставленных кипоинтов на query и database
                keypoint_xy_query = keypoints_query[keypoint_id_query]
                keypoint_xy_db = keypoints_db[keypoint_id_database]
                
                # получаем глубины данных кипоинтов
                depth_keypoint_query = depth_query[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
                depth_keypoint_db = depth_db[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]
                
                # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
                if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
                    continue
                    
                # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
                x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query
                
                # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
                kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
                kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
                kp_right.pt = (x_right, keypoint_xy_query[1])
                meas = Measurement(Measurement.Type.STEREO,
                                   Measurement.Source.TRIANGULATION,
                                   [kp_left, kp_right])
                
                #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
                # с кипоинтом на query image
                # Получаем 3D координату ключевой точки на database image
                global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
                                                      fx, fy, cx, cy, \
                                                      translation, hdf5_file_db['quat_base'][num_db])
                mappoint = MapPoint(global_xyz_of_keypoint)
                # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
                # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
                meas.mappoint = mappoint
                measurements.append(meas)
                
                # Помечаем этот кипоинт на query image тем, что в дальнейшем его не нужно рассматривать - его уже
                # добавили в измерения
                all_matched_keypoints_ids[keypoint_id_query] = True
    else:
        for i, m in enumerate(measurements):
            optimizer.add_point(i, m.mappoint.position, fixed=True)
            optimizer.add_edge(0, i, 0, m)
        optimizer.optimize(10)
        result = optimizer.get_pose(0)
        data['g2o']['cam_pose'] = result
    
        pose_estimated = np.eye(4)
        pose_query = np.eye(4)
    
#     r = R.from_quat([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
    
#     # Now get pose from g2o optimization 
#     matrix = result.matrix()[:3,:3]
#     pose_estimated[:3, :3] = matrix
#     pose_estimated[:3, 3] = result.position()
#     print("Estimated translation:")
#     print(result.position())
#     print(matrix)

    # Now get pose from NetVLAD
#     pose_estimated[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
#     pose_estimated[:3, 3] = hdf5_file_db['gps_base'][num_db]
    
#     orientation_quat_xyzw = [hdf5_file_query['quat'][num_query][1], hdf5_file_query['quat'][num_query][2],\
#                             hdf5_file_query['quat'][num_query][3], hdf5_file_query['quat'][num_query][0]]
#     r = R.from_quat(orientation_quat_xyzw)
#     matrix = r.as_matrix()
        pose_query[:3, :3] = quaternion_to_rotation_matrix(hdf5_file_query['quat'][num_query])
        pose_query[:3, 3] = hdf5_file_query['gps'][num_query]
        data['query']['cam_pose'] = pose_query 
#     print("GT translation:")
#     print(hdf5_file_query['gps'][num_query])
        error_pose = np.linalg.inv(pose_estimated) @ pose_query
        dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(error_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_error = abs(90 - abs(angle_error-90))
    
#     if num_point_db != num_point_query:
        
    
#     if angle_error > max_angle_error:
#         max_angle_error = angle_error
#         print(query_image)
#         print(db_image)
#         print(max_angle_error)
#         print("len of measurements = {}".format(len(measurements)))
#         print()
#         print()
#     print("angle error = {.f}", angle_error)
#     print("dist error = {.f}", dist_error)
#         print(db_image)
#         print(query_image)
#         print()

#     results[query_image] = {
#         'xyz': list(map(float, result.position())),
#         'quaternion_xyzw': list(map(float,r.as_quat()))
#     }
#     if angle_error < 20 and dist_error < 5:
#         metrics["(5m, 20°)"] += 1
#     if angle_error < 10 and dist_error < 1:
#         metrics["(1m, 10°)"] += 1
#     if angle_error < 5 and dist_error < 0.5:
#         metrics["(0.5m, 5°)"] += 1
#     if angle_error < 2 and dist_error < 0.25:
#         metrics["(0.25m, 2°)"] += 1
        if dist_error < 5:
            metrics["(5m, 20°)"] += 1
        if dist_error < 1:
            metrics["(1m, 10°)"] += 1
        if dist_error < 0.5:
            metrics["(0.5m, 5°)"] += 1
        if dist_error < 0.25:
            metrics["(0.25m, 2°)"] += 1
#     break
for key in metrics.keys():
    metrics[key] = metrics[key] / 950
# out.release()

In [ ]:
out = cv2.VideoWriter('/home/NetVLAD_wrong.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, (256, 256))

In [85]:
metrics

{'(5m, 20°)': 0.8547368421052631,
 '(1m, 10°)': 0.035789473684210524,
 '(0.5m, 5°)': 0.0,
 '(0.25m, 2°)': 0.0}

## Getting table of localizations

In [52]:
def quaternion_to_rotation_matrix(quaternion_wxyz):
    r = R.from_quat([quaternion_wxyz[1], quaternion_wxyz[2], quaternion_wxyz[3], quaternion_wxyz[0]])
    matrix = r.as_matrix()
    matrix[:3,2] = -matrix[:3,2]
    matrix[:3,1] = -matrix[:3,1]
    return matrix

In [53]:
def get_point_3d(x, y, depth, fx, fy, cx, cy, cam_center_world, R_world_to_cam, w_in_quat_first = True):
    if depth <= 0:
        return 0
    new_x = (x - cx)*depth/fx
    new_y = (y - cy)*depth/fy
    new_z = depth
    coord_3D_world_to_cam = np.array([new_x, new_y, new_z], float)
    if len(R_world_to_cam) == 4:
        if w_in_quat_first:
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
        else:
            R_world_to_cam = [R_world_to_cam[3], R_world_to_cam[0], R_world_to_cam[1], R_world_to_cam[2]]
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
    coord_3D_cam_to_world = np.matmul(matrix, coord_3D_world_to_cam) + cam_center_world
    return coord_3D_cam_to_world

In [54]:
retrievals = parse_retrieval(loc_pairs)

In [61]:
path_to_images = str(images_path)
path_to_hdf5_datasets = '/datasets/Habitat/HPointLoc/1LXtFkjw3qL_point0'

In [62]:
distances = np.array([0, 0.25, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 50])
angles = np.array([0, 1, 2, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 120, 150, 180], dtype=float)

table = {}
for distance in distances:
    if distance == 0:
        continue
    for angle in angles:
        if angle == 0:
            continue
        table[str(distance)+" m and "+ str(angle) + " deg"] = {
            'tp'           : 0,
            'all'           : 0,
            'queries-dbs' : {}
        }

In [63]:
keypoints_file = h5py.File(str(outputs / feature_filename), 'r')
match_filename = 'feats-superpoint-n4096-r1024-nms4_matches-superglue_HF_Net_qdb_qdb.h5'
match_file = h5py.File(str(outputs / match_filename), 'r')

In [68]:
images_path = dataset / 'extracted_HPointLoc/1LXtFkjw3qL_point0/images'
path_to_images = str(images_path)

In [72]:
# optimizer = BundleAdjustment()

for filename1 in tqdm(sorted(os.listdir(path_to_images))):
    query1_filename = filename1.rstrip('.png')
    hdf5_filename_query1 = '_'.join(query1_filename.split('_')[:2]) + '.hdf5'
    hdf5_file_query1 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query1), 'r')
    num_query1 = int(query1_filename.split('_')[-1])
    
    is_database = ''
    if query1_filename.find('database') != -1:
        is_database = '_base'
    
    depth_query1 = 10*hdf5_file_query1['depth'+is_database][num_query1]
    
    translation = hdf5_file_query1['gps'+is_database][num_query1]
    orientation = quaternion_to_rotation_matrix(hdf5_file_query1['quat'+is_database][num_query1])
    pose_44_query1 = np.eye(4)
    pose_44_query1[:3,:3] = orientation
    pose_44_query1[:3,3] = translation
    
    # Заполняем таблицу метрик    
    for filename2 in sorted(os.listdir(path_to_images)):
        if filename1 == filename2:
            continue
        query2_filename = filename2.rstrip('.png')
        hdf5_filename_query2 = '_'.join(query2_filename.split('_')[:2]) + '.hdf5'
        hdf5_file_query2 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query2), 'r')
        
        num_query2 = int(query2_filename.split('_')[-1])
        
        is_database = ''
        if query2_filename.find('database') != -1:
            is_database = '_base'
        
        translation = hdf5_file_query2['gps'+is_database][num_query2]
        orientation = quaternion_to_rotation_matrix(hdf5_file_query2['quat'+is_database][num_query2])
        pose_44_query2 = np.eye(4)
        pose_44_query2[:3,:3] = orientation
        pose_44_query2[:3,3] = translation
        
        diff_pose = np.linalg.inv(pose_44_query2) @ pose_44_query1
        dist_diff = np.sum(diff_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(diff_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_diff = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_diff = abs(180 - abs(angle_diff-180))
        
        num_dist =  [i for i in range(1, len(distances)-1) \
                     if (distances[i-1] < dist_diff) and (distances[i] >= dist_diff)]
        
        num_angle = [i for i in range(1, len(angles)-1) \
                     if (angles[i-1] < angle_diff) and (angles[i] >= angle_diff)]
        
        # Если не входит в таблицу, то пропускаем эту пару
        if len(num_dist) == 0 or len(num_angle) == 0:
            continue
        
        if query1_filename in table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'].keys():
            table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'][query1_filename]["db_filenames"].append(query2_filename)
#             table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]["all"] += 1
        else:
            table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'][query1_filename] = {
                "db_filenames"   : [query2_filename]
            }
#             table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]["all"] += 1

100%|██████████| 1088/1088 [18:28<00:00,  1.02s/it]


In [49]:
# import faiss
# from sklearn.metrics.pairwise import cosine_similarity
import torch
from hloc.utils.base_model import dynamic_load
from hloc import matchers
confs = {
    'superglue': {
        'output': 'matches-superglue',
        'model': {
            'name': 'superglue',
            'weights': 'outdoor',
            'sinkhorn_iterations': 50,
        },
    },
    'NN': {
        'output': 'matches-NN-mutual-dist.7',
        'model': {
            'name': 'nearest_neighbor',
            'mutual_check': True,
            'distance_threshold': 0.7,
        },
    }
}

In [165]:
out = cv2.VideoWriter('/home/NetVLAD_wrong.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, (256, 256))
output_folder_for_images = '/home/for_video'
os.makedirs(output_folder_for_images, exist_ok=True)
save_images = True

import PIL
from PIL import Image
from PIL import ImageFont, ImageDraw
from sklearn.metrics.pairwise import cosine_similarity


fontsize = 13
font = ImageFont.truetype("DejaVuSans.ttf", fontsize)

count = 0

In [478]:
images_path = '/datasets/Habitat/1LXtFkjw3qL_point0/images/'

In [479]:
save_images = False

In [485]:
num_keypoints = 4096
path_to_global_descriptors = '/datasets/Habitat/HF_Net_output/global_descriptors'

conf = confs['superglue']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Model = dynamic_load(matchers, conf['model']['name'])
model = Model(conf['model']).eval().to(device)

count = 0

for cell in tqdm(table.keys()):
    if len(table[cell]['queries-dbs'].keys()) == 0:
        continue
        
#     if cell != "2.0 m and 10.0 deg":
#         continue
        
    for query_image in table[cell]['queries-dbs'].keys():
        
        # RUN global descriptor retrieval and find top-1 retrieval image
    
        dbFeat = []

        with open(os.path.join(path_to_global_descriptors, query_image+'.npy'), 'rb') as f:
            descriptor_query = np.load(f)
        qFeat = np.empty((1, num_keypoints))
        qFeat[0,:] = descriptor_query
        qFeat = qFeat.astype('float32')
        faiss_index = faiss.IndexFlatL2(num_keypoints)
        faiss_index.add(qFeat)
    
        measurements = []
        optimizer.clear()
        for db_image in table[cell]['queries-dbs'][query_image]["db_filenames"]:
        
            with open(os.path.join(path_to_global_descriptors, db_image+'.npy'), 'rb') as f:
                descriptor2 = np.load(f)
            dbFeat.append(descriptor2)
        
        dbFeat_temp = np.empty((len(dbFeat), num_keypoints))

        for i in range(len(dbFeat)):
            dbFeat_temp[i,:] = dbFeat[i]
    
        dbFeat = dbFeat_temp
    
        dbFeat = dbFeat.astype('float32')
        distances, predictions = faiss_index.search(dbFeat, 1)
        num_db = predictions[0][0]
        db_image = table[cell]['queries-dbs'][query_image]["db_filenames"][num_db]
        image_1 = query_image
        image_2 = db_image
        
        dbfeature = dbFeat[num_db]
        qfeature  = qFeat[0]
        dbfeature = np.reshape(dbfeature, (1,-1))
        qfeature = np.reshape(qfeature, (1,-1))
        simil_score = cosine_similarity(dbfeature, qfeature)[0][0]
    
        # Run SuperPoint + SuperGlue + G2O
    
        query1_filename = image_1
        is_database = ''
        if query1_filename.find('database') != -1:
            is_database = '_base'
        hdf5_filename_query1 = '_'.join(query1_filename.split('_')[:2]) + '.hdf5'
        hdf5_file_query1 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query1), 'r')
        num_query1 = int(query1_filename.split('_')[-1])
        depth_query1 = 10*hdf5_file_query1['depth'+is_database][num_query1]
        translation = hdf5_file_query1['gps'+is_database][num_query1]
        orientation = quaternion_to_rotation_matrix(hdf5_file_query1['quat'+is_database][num_query1])
        pose_44_query1 = np.eye(4)
        pose_44_query1[:3,:3] = orientation
        pose_44_query1[:3,3] = translation
        keypoints_query = keypoints_file[query1_filename]['keypoints'].__array__()
    
        db_filename = image_2
        hdf5_filename_db = '_'.join(db_filename.split('_')[:2]) + '.hdf5'
        hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
        num_db = int(db_filename.split('_')[-1])
        is_database = ''
        if db_filename.find('database') != -1:
            is_database = '_base'
        translation = hdf5_file_db['gps'+is_database][num_db]
        orientation = quaternion_to_rotation_matrix(hdf5_file_db['quat'+is_database][num_db])
        pose_44_db = np.eye(4)
        pose_44_db[:3,:3] = orientation
        pose_44_db[:3,3] = translation
        depth_db = 10*hdf5_file_db['depth'+is_database][num_db]
        keypoints_db = keypoints_file[db_filename]['keypoints'].__array__()
    
        pose = g2o.Isometry3d(pose_44_db)
        optimizer.add_pose(0, pose, cam, fixed=False)
    
        retrieved_image = image_2
        hdf5_file_retrieved = hdf5_file_db
        num_retrieved = num_db
        depth_retrieved = depth_db
        
        # Extract matches
        
        data = {}
        feats0, feats1 = keypoints_file[query_image], keypoints_file[db_image]
        for k in feats1.keys():
            data[k+'0'] = feats0[k].__array__()
        for k in feats1.keys():
            data[k+'1'] = feats1[k].__array__()
        data = {k: torch.from_numpy(v)[None].float().to(device)
                for k, v in data.items()}
        data['image0'] = torch.empty((1, 1,)+tuple(feats0['image_size'])[::-1])
        data['image1'] = torch.empty((1, 1,)+tuple(feats1['image_size'])[::-1])
        
        pred = model(data)
        
        matches_db_keypoints_ids = pred['matches0'][0].cpu().short().numpy()
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        
        # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
        for num_kp, keypoint_id in enumerate(matched_keypoints_query_ids):
            if keypoint_id == -1:
                continue
            # затем получаем индекс сопоставленного кипоинта на database image
            keypoint_id_query = keypoint_id
            keypoint_id_database = matches_db_keypoints_ids[keypoint_id]
                
            # получаем координаты x,y для сопоставленных кипоинтов на query и database
            keypoint_xy_query = keypoints_query[keypoint_id_query]
            keypoint_xy_db = keypoints_db[keypoint_id_database]
                
            # получаем глубины данных кипоинтов
            depth_keypoint_query = depth_query1[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
            depth_keypoint_db = depth_retrieved[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]
                
            # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
            if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
                continue
                    
            # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
            x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query
                
            # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
            kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
            kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
            kp_right.pt = (x_right, keypoint_xy_query[1])
            meas = Measurement(Measurement.Type.STEREO,
                               Measurement.Source.TRIANGULATION,
                               [kp_left, kp_right])
                
            #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
            # с кипоинтом на query image
            # Получаем 3D координату ключевой точки на database image
            global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
                                     fx, fy, cx, cy, \
                                     translation, hdf5_file_retrieved['quat'+is_database][num_retrieved])
            mappoint = MapPoint(global_xyz_of_keypoint)
            # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
            # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
            meas.mappoint = mappoint
            measurements.append(meas)
                
        for i, m in enumerate(measurements):
            optimizer.add_point(i, m.mappoint.position, fixed=True)
            optimizer.add_edge(0, i, 0, m)
        optimizer.optimize(10)
        result = optimizer.get_pose(0)
    
        pose_estimated = np.eye(4)
        # Now get pose from g2o optimization 
        matrix = result.matrix()[:3,:3]
        pose_estimated[:3, :3] = matrix
        pose_estimated[:3, 3] = result.position()
    
        error_pose = np.linalg.inv(pose_estimated) @ pose_44_query1
        dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(error_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_error = abs(180 - abs(angle_error - 180))
    
        table[cell]['all'] += 1
        if angle_error < 5 and dist_error < 0.5:
            table[cell]['tp'] += 1
            
        table[cell]['queries-dbs'][query_image]["estimated_pose"] = pose_estimated.tolist()
        table[cell]['queries-dbs'][query_image]["retrieved_image"] = retrieved_image
        table[cell]['queries-dbs'][query_image]["retrival_score"] = simil_score
        table[cell]['queries-dbs'][query_image]["num_matches"] = float(simil_score)
            
        if dist_error > 0.5 or angle_error > 5 and save_images:
            
            new_image = Image.new('RGB',(2*256, 256), (250,250,250))

            db_image1 = db_image
        
            image1 = Image.open(str(images_path / Path(query_image))+'.png')
            image2 = Image.open(str(images_path / Path(db_image1))+'.png')
        
#             metric1 = list(retrievals[query_image+'.png'][0].values())[0]
        
            new_image.paste(image1,(0,0))
            new_image.paste(image2,(255,0))

            draw = ImageDraw.Draw(new_image)
        
            draw.text((0, 0), query_image,(255,255,255), font=font)
            draw.text((image1.size[0], 0),"{}\n{}".format(db_image.rstrip('.png'), round(float(simil_score),3)),(255,255,255),font=font)

            new_image.save('{}/{:04}.jpg'.format(output_folder_for_images, count),'JPEG')
            count += 1
            
#         print(table[cell])
#         print(dist_error)
#         print(angle_error)
#     break

  0%|          | 0/176 [00:00<?, ?it/s]

Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 176/176 [3:39:27<00:00, 74.82s/it]   


In [482]:
# imgage_array = []

# for filename in sorted(os.listdir(output_folder_for_images)):
#     img = cv2.imread(os.path.join(output_folder_for_images, filename))
#     height, width, layers = img.shape
#     size = (width,height)
#     imgage_array.append(img)

# out = cv2.VideoWriter('/home/video_Habitat.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)

# for i in range(len(imgage_array)):
#     cv2.waitKey(10000)
#     out.write(imgage_array[i])
# out.release()

In [492]:
import json
with open('table.json', 'w') as outfile:
    json.dump(table, outfile)

In [442]:
path_to_global_descriptors = '/datasets/Habitat/HF_Net_output/global_descriptors'

# image_filename1 = '1LXtFkjw3qL_point13_query_0002'
# image_filename2 = '1LXtFkjw3qL_point0_query_0029'

image_filename1 = '1LXtFkjw3qL_point12_query_0007'
image_filename2 = '1LXtFkjw3qL_point16_query_0038'

with open(os.path.join(path_to_global_descriptors, image_filename1+'.npy'), 'rb') as f:
    descriptor1 = np.load(f)
    
with open(os.path.join(path_to_global_descriptors, image_filename2+'.npy'), 'rb') as f:
    descriptor2 = np.load(f)

descriptor1 = np.reshape(descriptor1, (1,-1))
descriptor2 = np.reshape(descriptor2, (1,-1))
print(cosine_similarity(descriptor1, descriptor2)[0][0])
print(np.sqrt(np.sum((descriptor1-descriptor2)**2, axis=1)))

0.6093401
[0.88392293]


In [ ]:
    # Получаем предсказание image retrival
    retrieved_image = [key for key, value in retrievals[query1_filename+'.png'][0].items()][0]
    if retrieved_image == query1_filename+'.png':
        retrieved_image = [key for key, value in retrievals[query1_filename+'.png'][1].items()][0]
        
    # Выполняем подготовку для потимизации позы
    measurements = []
    optimizer.clear()
        
    hdf5_filename_retrieved = '_'.join(retrieved_image.split('_')[:2]) + '.hdf5'
    hdf5_file_retrieved = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_retrieved), 'r')
        
    num_retrieved = int(retrieved_image.rstrip('.png').split('_')[-1])
        
    is_database = ''
    if retrieved_image.find('database') != -1:
        is_database = '_base'
        
    depth_retrieved = 10*hdf5_file_retrieved['depth'+is_database][num_retrieved]
        
    translation = hdf5_file_retrieved['gps'+is_database][num_retrieved]
    orientation = quaternion_to_rotation_matrix(hdf5_file_retrieved['quat'+is_database][num_retrieved])
    pose_retrieved = np.eye(4)
    pose_retrieved[:3,:3] = orientation
    pose_retrieved[:3,3] = translation
    pose = g2o.Isometry3d(pose_retrieved)
    optimizer.add_pose(0, pose, cam, fixed=False)
        
    keypoints_db = keypoints_file[retrieved_image.rstrip('.png')]['keypoints'].__array__()
    keypoints_query = keypoints_file[query1_filename.rstrip('.png')]['keypoints'].__array__()
#     print(len(keypoints_query))
#     print(len(keypoints_db))
        
        # matches_db_keypoints_ids - массив из индексов кипоинтов на database image, которые совпадают с кипоинтом, индекс которого равен положению в массиве        
        # matched_keypoints_query_ids - массив из индексов кипоинтов на query image, которые имеют сопоставление
    inverse = False
    if '_'.join([query1_filename,retrieved_image.rstrip('.png')]) in match_file.keys():
        matches_db_keypoints_ids = match_file['_'.join([query1_filename, retrieved_image.rstrip('.png')])]['matches0'].__array__()
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
    elif '_'.join([retrieved_image.rstrip('.png'),query1_filename]) in match_file.keys():
        matched_keypoints_query_ids = match_file['_'.join([retrieved_image.rstrip('.png'), query1_filename])]['matches0'].__array__()
        matches_db_keypoints_ids = np.where(matched_keypoints_query_ids > -1)[0]
        inverse = True
    else:
        print('pair does not exist')
        print('_'.join([query1_filename,retrieved_image.rstrip('.png')]))
        
    # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
    for num_kp, keypoint_id in enumerate(matched_keypoints_query_ids):
        if keypoint_id == -1:
            continue
#         print(inverse)
#         print(matched_keypoints_query_ids)
#         print(matches_db_keypoints_ids)
#         print()
        # затем получаем индекс сопоставленного кипоинта на database image
        if not(inverse):
            keypoint_id_query = keypoint_id
            keypoint_id_database = matches_db_keypoints_ids[keypoint_id]
        else:
            keypoint_id_database = num_kp
            keypoint_id_query = keypoint_id
                
        # получаем координаты x,y для сопоставленных кипоинтов на query и database
        keypoint_xy_query = keypoints_query[keypoint_id_query]
        keypoint_xy_db = keypoints_db[keypoint_id_database]
                
        # получаем глубины данных кипоинтов
        depth_keypoint_query = depth_query1[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
        depth_keypoint_db = depth_retrieved[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]
                
        # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
        if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
            continue
                    
        # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
        x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query
                
        # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
        kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
        kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
        kp_right.pt = (x_right, keypoint_xy_query[1])
        meas = Measurement(Measurement.Type.STEREO,
                          Measurement.Source.TRIANGULATION,
                          [kp_left, kp_right])
                
        #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
        # с кипоинтом на query image
        # Получаем 3D координату ключевой точки на database image
        global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
                                            fx, fy, cx, cy, \
                                            translation, hdf5_file_retrieved['quat'+is_database][num_retrieved])
        mappoint = MapPoint(global_xyz_of_keypoint)
        # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
        # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
        meas.mappoint = mappoint
        measurements.append(meas)
                
    for i, m in enumerate(measurements):
        optimizer.add_point(i, m.mappoint.position, fixed=True)
        optimizer.add_edge(0, i, 0, m)
    optimizer.optimize(10)
    result = optimizer.get_pose(0)
    
    pose_estimated = np.eye(4)
    pose_query = np.eye(4)
    
    # Now get pose from g2o optimization 
    matrix = result.matrix()[:3,:3]
    pose_estimated[:3, :3] = matrix
    pose_estimated[:3, 3] = result.position()
    
    error_pose = np.linalg.inv(pose_estimated) @ pose_44_query1
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(90 - abs(angle_error-90))

In [68]:
# RANSAC

results = {}
metrics = {
    "(5m, 20°)": 0,
    "(1m, 10°)": 0,
    "(0.5m, 5°)": 0,
    "(0.25m, 2°)": 0
}
for query_image, db_images in tqdm(retrievals.items()):
    keypoints_2D = []
    keypoints_3D = []
    hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
    hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
    
    num_query = int(query_image.split('_')[-1])
    
    keypoints_query = keypoints_file[query_image]['keypoints'].__array__()
    all_matched_keypoints_ids = np.full((len(keypoints_query)), False)
    
    for num in range(len(db_images)):
        if num >= top_num_db_image:
            break
        db_image = db_images[num]
        hdf5_filename_db = '_'.join(db_image.split('_')[:2]) + '.hdf5'
        hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
        
        num_db = int(db_image.split('_')[-1])
        
        depth_db = 10*hdf5_file_db['depth_base'][num_db]
        
        translation = hdf5_file_db['gps_base'][num_db]
        orientation_matrix_44 = quaternion_to_rotation_matrix(hdf5_file_db['quat_base'][num_db])
        pose_44 = np.eye(4)
        pose_44[:3,:3] = orientation_matrix_44
        pose_44[:3,3] = translation
        
        matches_db_keypoints_ids = match_file['_'.join([query_image,db_image])]['matches0'].__array__()
        matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        for keypoint_id_query in matched_keypoints_query_ids:
            if all_matched_keypoints_ids[keypoint_id_query] == False:
                keypoint = keypoints_query[keypoint_id_query]
                depth_keypoint = depth_db[int(keypoint[1]), int(keypoint[0])][0]
                if depth_keypoint <= 0:
                    continue
                global_xyz_of_keypoint = get_point_3d(keypoint[0], keypoint[1], depth_keypoint, \
                                                      fx, fy, cx, cy, \
                                                      translation, hdf5_file_db['quat_base'][num_db])
                keypoint += 0.5
                keypoints_2D.append(list(keypoint))
                keypoints_3D.append(list(global_xyz_of_keypoint))
                all_matched_keypoints_ids[keypoint_id_query] = True           
#     ret = pycolmap.absolute_pose_estimation(keypoints_2D, keypoints_3D, cfg, 12)
#     print(ret['success'])
#     print(ret['tvec'])
#     print(yaw_pitch_roll_from_quaternion(list(ret['qvec']), w_first=True))
#     break
#     r = R.from_matrix(result.matrix()[:3,:3])
#     yaw_pitch_roll = r.as_euler('yxz', degrees=True)
    orientation_quat_wxyz = hdf5_file_db['quat_base'][num_db]
#     results[query_image] = {
#         'xyz': list(map(float,translation)),
#         'quaternion_xyzw': list([float(orientation[1]), float(orientation[2]), float(orientation[3]), float(orientation[0])])
#     }
#     r = R.from_quat(list([orientation[1], orientation[2], orientation[3], orientation[0]]))
#     yaw_pitch_roll = r.as_euler('yxz', degrees=True)
#     print(query_image)
#     print("NetVLAD")
#     print([orientation[1], orientation[2], orientation[3], orientation[0]])
#     print(translation)
#     print(yaw_pitch_roll)
#     print("query")

    pose_estimated = np.eye(4)
    pose_query = np.eye(4)
    
    r = R.from_quat([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
#     print([orientation_quat_wxyz[1], orientation_quat_wxyz[2], orientation_quat_wxyz[3], orientation_quat_wxyz[0]])
    pose_estimated[:3, :3] = r.as_matrix()
    pose_estimated[:3, 3] = translation
    
    orientation_quat_xyzw = [hdf5_file_query['quat'][num_query][1], hdf5_file_query['quat'][num_query][2],\
                            hdf5_file_query['quat'][num_query][3], hdf5_file_query['quat'][num_query][0]]
    r = R.from_quat(orientation_quat_xyzw)
    pose_query[:3, :3] = r.as_matrix()
    pose_query[:3, 3] = hdf5_file_query['gps'][num_query]
    error_pose = np.linalg.inv(pose_estimated) @ pose_query
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = R.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
    angle_error = abs(90 - abs(angle_error-90))
    
    if dist_error < 5: #and angle_error < 20 and dist_error < 5:
        metrics["(5m, 20°)"] += 1
    if dist_error < 1: #and angle_error < 10:
        metrics["(1m, 10°)"] += 1
    if dist_error < 0.5: #and angle_error < 5:
        metrics["(0.5m, 5°)"] += 1
    if dist_error < 0.25: #and angle_error < 2:
        metrics["(0.25m, 2°)"] += 1
for key in metrics.keys():
    metrics[key] = metrics[key] / 950

 13%|█▎        | 120/950 [00:02<00:16, 49.92it/s]


KeyboardInterrupt: 

In [ ]:
metrics

In [ ]:
with open(str(outputs / 'localization_results.json'), 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
hdf5_filename_query = '_'.join(query_image.split('_')[:2]) + '.hdf5'
hdf5_file_query = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query), 'r')
num_query = int(query_image.split('_')[-1])
translation_query = hdf5_file_query['gps'][num_query]
orientation = quaternion_to_rotation_matrix(hdf5_file_query['quat'][num_query])
r = R.from_matrix(orientation)
yaw_pitch_roll = r.as_euler('yxz', degrees=True)
print(translation_query)
print(yaw_pitch_roll)

In [ ]:
len(results.keys())